In [2]:
import pandas as pd

#hello worl from new mac book air m3

# List of file paths
file_paths = ['infant_mortality_rate.csv', 'total_fertility_rate.csv']

# List to store DataFrames
dfs = []

# Read each CSV file into a DataFrame
for file_path in file_paths:
    df = pd.read_csv(file_path)
    dfs.append(df)

# Combine the DataFrames
combined_df = pd.concat(dfs)

# Handle missing or duplicate data
combined_df.drop_duplicates(inplace=True)
combined_df.fillna(0, inplace=True)  # Replace NaN values with 0

# Display combined DataFrame
print(combined_df.head())


  Country or Area  Year(s) Variant   Value
0     Afghanistan     2101  Medium  0.0000
1     Afghanistan     2100  Medium  7.4881
2     Afghanistan     2099  Medium  7.6078
3     Afghanistan     2098  Medium  7.7567
4     Afghanistan     2097  Medium  7.8799
